In [51]:
import pandas as pd
import numpy as np

fifa = pd.read_csv('../data/raw/FIFA_players_21.csv')


In [52]:
fifa = fifa.drop(columns=['defending_marking','loaned_from'])

In [53]:
print(fifa.columns.tolist())

['sofifa_id', 'player_url', 'short_name', 'long_name', 'age', 'dob', 'height_cm', 'weight_kg', 'nationality', 'club_name', 'league_name', 'league_rank', 'overall', 'potential', 'value_eur', 'wage_eur', 'player_positions', 'preferred_foot', 'international_reputation', 'weak_foot', 'skill_moves', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'player_tags', 'team_position', 'team_jersey_number', 'joined', 'contract_valid_until', 'nation_position', 'nation_jersey_number', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed', 'gk_positioning', 'player_traits', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_

In [ ]:
# fifa['value_eur']=np.where(fifa['value_eur'] > 10000000,np.log10(fifa['value_eur']),fifa['value_eur'])

In [54]:
fifa['value_eur'].head()

0    67500000
1    46000000
2    75000000
3    80000000
4    90000000
Name: value_eur, dtype: int64